In [ ]:
import collections
import functools
import http
import json
import logging
import os
import urllib

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import seaborn as sns
from matplotlib_venn import venn3, venn3_circles
from rdkit import Chem
from tqdm.notebook import tqdm

In [ ]:
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='serif')
sns.set_palette('Set1')
sns.set_context('paper', font_scale=1.3)

tqdm.pandas(leave=False)

logging.basicConfig(format='%(asctime)s [%(levelname)s/%(processName)s] '
                           '%(module)s.%(funcName)s : %(message)s',
                    level=logging.INFO)

In [ ]:
@functools.lru_cache(None)
def pubchem_name_to_inchikey(name):
    try:
        with urllib.request.urlopen(
                f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/'
                f'{name}/property/inchikey/TXT') as f_url:
            return f_url.read().decode().strip().split()[0]
    except urllib.error.HTTPError:
        return None

    
@functools.lru_cache(None)
def smiles_to_inchikey(smiles):
    try:
        mol = Chem.rdmolfiles.MolFromSmiles(smiles)
    except ValueError:
        mol = None
    return Chem.rdinchi.MolToInchiKey(mol) if mol is not None else None


@functools.lru_cache(None)
def inchikey_to_subclass(inchikey):
    try:
        with urllib.request.urlopen(f'https://gnps-classyfire.ucsd.edu/'
                                    f'entities/{inchikey}.json') \
                as f_url:
            return json.loads(f_url.read())['subclass']['chemont_id']
    except (urllib.error.HTTPError, AttributeError, TypeError, KeyError):
        return None

In [ ]:
data_dir = '../data/deconvolution_comparison'
compounds = {
    'Metabolomics Workbench ST001154':
        set(pd.read_csv(os.path.join(
            data_dir, 'Metabolomics Workbench ST001154.csv'))['InChiKey']),
    'LECO beer aging':
        set(pd.read_csv(os.path.join(data_dir, 'LECO beer aging.csv'))
            ['Name'].apply(pubchem_name_to_inchikey))
}
subclasses = {dataset: set([inchikey_to_subclass(inchikey)
                            for inchikey in comp])
              for dataset, comp in compounds.items()}
for sc in subclasses.values():
    sc.discard(None)

In [ ]:
task_ids = {
    'Metabolomics Workbench ST001154': {
        'MSHub': '4f2de61a', 'MZmine': '021f4818', 'MS-DIAL': '16d202bd'},
    'LECO beer aging': {
        'MSHub': '35445438', 'MZmine': '37971f05', 'MS-DIAL': 'dd21a47e'}}

identifications = collections.defaultdict(
    lambda: collections.defaultdict(dict))
for top in (1, 10):
    for dataset, tools in task_ids.items():
        logging.info('Process %s', dataset)
        for tool, task_id in tools.items():
            logging.info('%s identifications', tool)
            # Read the GNPS task file.
            usecols = ['#Scan#', 'INCHI', 'Smiles', 'MQScore', 'SharedPeaks']
            if tool == 'MSHub':
                usecols.append('Balance_score(percentage)')
            ids = (
                pd.read_csv(
                    os.path.join(
                        data_dir, f'MOLECULAR-LIBRARYSEARCH-GC-{task_id}-'
                        f'view_all_annotations_DB-main.tsv'),
                    sep='\t', usecols=usecols, skipinitialspace=True)
                .dropna()
                .sort_values(['#Scan#', 'MQScore'], ascending=[True, False])
                .groupby('#Scan#').head(top))
            # Require a minimum cosine score of 0.8 and
            # more than 10 matched peaks.
            ids = ids[(ids['MQScore'] > 0.8) & (ids['SharedPeaks'] > 10)]
            # Use only the top match per compound.
            ids = (ids.sort_values('MQScore', ascending=False)
                   .drop_duplicates('INCHI'))
            # Get InChIKeys and subclasses for all identifications.
            ids['InChIKey'] = ids['Smiles'].apply(smiles_to_inchikey)
            ids['subclass'] = \
                ids['InChIKey'].progress_apply(inchikey_to_subclass)
            ids.dropna()
            
            identifications[top][dataset][tool] = ids

In [ ]:
tops, datasets, tools, match_type, perc_matched = [], [], [], [], []
for top, dataset_ids in identifications.items():
    for dataset, tool_ids in dataset_ids.items():
        for tool, ids in tool_ids.items():
            tops.append(top)
            datasets.append(dataset)
            tools.append(tool)
            match_type.append('Direct match')
            perc_matched.append(
                len(set(ids['InChIKey']) & compounds[dataset])
                / len(compounds[dataset]))
            tops.append(top)
            datasets.append(dataset)
            tools.append(tool)
            match_type.append('Subclass match')
            perc_matched.append(
                len(set(ids['subclass']) & subclasses[dataset])
                / len(subclasses[dataset]))

matched = pd.DataFrame({'Dataset': datasets, 'Tool': tools, 'Top': tops,
                        'Type': match_type,
                        'Compounds identified': perc_matched})

g = sns.catplot(x='Tool', y='Compounds identified', hue='Type', data=matched,
                row='Dataset', col='Top', kind='bar', height=4, aspect=1.5,
                legend=False)

for ax in g.axes.ravel():
    ax.set_ylim(0, 1)
    ax.yaxis.set_major_formatter(mticker.PercentFormatter(1))

g.axes[0, -1].legend(loc='upper center', bbox_to_anchor=(0, 1.35),
                     title='Type', fontsize='medium', ncol=2)

sns.despine()

plt.savefig('deconvolution_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
height = 8
width = height
num_col = len(identifications)
num_row = len(identifications[1])
fig, axes = plt.subplots(
    num_row, num_col, figsize=(width * num_col, height * num_row))

for col, (top, top_ids) in enumerate(identifications.items()):
    for row, (dataset, dataset_ids) in enumerate(top_ids.items()):
        ax = axes[row, col]
        
        sets = []
        for tool, tool_ids in dataset_ids.items():
            sets.append(set(tool_ids[tool_ids['InChIKey'].isin(
                compounds[dataset])]['InChIKey']))
        v = venn3(sets, set_labels=dataset_ids.keys(), ax=ax)
        c = venn3_circles(sets, linewidth=1.0, ax=ax)
        for text in v.subset_labels:
            if text is not None:
                text.set_fontsize('large')

        ax.set_title(f'Dataset = {dataset} | Top = {top}', fontsize='large')

plt.savefig('deconvolution_comparison_venn_direct_match.png',
            dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
height = 8
width = height
num_col = len(identifications)
num_row = len(identifications[1])
fig, axes = plt.subplots(
    num_row, num_col, figsize=(width * num_col, height * num_row))

for col, (top, top_ids) in enumerate(identifications.items()):
    for row, (dataset, dataset_ids) in enumerate(top_ids.items()):
        ax = axes[row, col]
        
        sets = []
        for tool, tool_ids in dataset_ids.items():
            sets.append(set(tool_ids[tool_ids['subclass'].isin(
                subclasses[dataset])]['subclass']))
        v = venn3(sets, set_labels=dataset_ids.keys(), ax=ax)
        c = venn3_circles(sets, linewidth=1.0, ax=ax)
        for text in v.subset_labels:
            if text is not None:
                text.set_fontsize('large')

        ax.set_title(f'Dataset = {dataset} | Top = {top}', fontsize='large')

plt.savefig('deconvolution_comparison_venn_subclass_match.png',
            dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
cosines

In [ ]:
cosines.set_index(order).sort_index()

In [ ]:
tops, datasets, tools, cosines = [], [], [], []
for top, top_ids in identifications.items():
    for dataset, dataset_ids in top_ids.items():
        for tool, tool_ids in dataset_ids.items():
            ids = tool_ids[tool_ids['InChIKey'].isin(compounds[dataset])]
            tops.extend([top] * len(ids))
            datasets.extend([dataset] * len(ids))
            tools.extend([tool] * len(ids))
            cosines.extend(ids['MQScore'].values)
min_balance_scores = [60, 80]
for min_balance_score in min_balance_scores:
    for top, top_ids in identifications.items():
        for dataset, dataset_ids in top_ids.items():
            ids = dataset_ids['MSHub']
            ids = ids[(ids['InChIKey'].isin(compounds[dataset])) &
                      (ids['Balance_score(percentage)'] > min_balance_score)]
            tops.extend([top] * len(ids))
            datasets.extend([dataset] * len(ids))
            tools.extend([f'MSHub\n(Balance score > {min_balance_score}%)']
                         * len(ids))
            cosines.extend(ids['MQScore'].values)

cosines = pd.DataFrame({'Top': tops, 'Dataset': datasets, 'Tool': tools,
                        'Cosine score': cosines})
# Custom sort to group MSHub results.
order = cosines['Tool'].replace({
    'MSHub': 0, 'MSHub\n(Balance score > 60%)': 1,
    'MSHub\n(Balance score > 80%)': 2, 'MZmine': 3, 'MS-DIAL': 4})
cosines = cosines.set_index(order).sort_index()

g = sns.catplot(x='Tool', y='Cosine score', hue='Top', data=cosines,
                row='Dataset', kind='violin', height=4, aspect=3,
                legend=False, cut=0, scale='count', inner='point')

g.axes[0, -1].legend(loc='upper center', bbox_to_anchor=(0.5, 1.35),
                     title='Top', fontsize='medium', ncol=2)

sns.despine()

plt.savefig('deconvolution_cosine_score.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
height = 6
width = height * 1.5
fig, ax = plt.subplots(figsize=(width, height))

tops, datasets, balance_scores = [], [], []
for top, top_ids in identifications.items():
    for dataset, dataset_ids in top_ids.items():
        ids = dataset_ids['MSHub']
        ids = ids[ids['InChIKey'].isin(compounds[dataset])]
        tops.extend([top] * len(ids))
        datasets.extend([dataset] * len(ids))
        balance_scores.extend(ids['Balance_score(percentage)'].values)

balance_scores = pd.DataFrame({'Top': tops, 'Dataset': datasets,
                               'Balance score': balance_scores})
sns.violinplot(x='Dataset', y='Balance score', hue='Top',
               data=balance_scores, cut=0, scale='count', inner='point')

ax.yaxis.set_major_formatter(mticker.PercentFormatter(100))

ax.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Top')

sns.despine()

plt.savefig('deconvolution_balance_score.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
logging.shutdown()